In [ ]:
# This code is used to analyze the schema of a CSV file and return SQL-like schema information.
# It reads the CSV file, gets the column information, and outputs the schema information as a string.
# The schema information is a string that contains the column name, the data type, and whether the column is nullable.
# The output is a string that can be used to create a table in a database.

In [ ]:
import pandas as pd
import os

def analyze_csv_schema(csv_file):
    """Analyze the schema of a CSV file and return SQL-like schema information"""
    try:
        # Read the CSV file
        df = pd.read_csv(csv_file)
        
        # Get column info
        schema_info = []
        for column in df.dtypes.items():
            col_name = column[0]
            # Map pandas dtypes to SQL-like types
            if 'int' in str(column[1]):
                sql_type = 'INTEGER'
            elif 'float' in str(column[1]):
                sql_type = 'FLOAT'
            elif 'datetime' in str(column[1]):
                sql_type = 'DATETIME'
            elif 'bool' in str(column[1]):
                sql_type = 'BOOLEAN'
            else:
                sql_type = 'VARCHAR'
            
            # Check if column has any null values
            nullable = 'NULL' if df[col_name].isnull().any() else 'NOT NULL'
            
            schema_info.append(f"{col_name} {sql_type} {nullable}")
        
        # Format schema output
        table_name = os.path.splitext(os.path.basename(csv_file))[0].upper()
        schema = f"CREATE TABLE {table_name} (\n    " + ",\n    ".join(schema_info) + "\n);"
        
        return schema
    
    except Exception as e:
        return f"Error analyzing schema: {str(e)}"

In [2]:
# Example usage:
schema = analyze_csv_schema('final_forecast_sheet.csv')
print(schema)

CREATE TABLE FINAL_FORECAST_SHEET (
    id INTEGER NOT NULL,
    DC_FC_Assets_Type VARCHAR NULL,
    DC_FC_Assets_Name VARCHAR NOT NULL,
    SQL_FC_Account_ID VARCHAR NOT NULL,
    SQL_Heading_Sequence INTEGER NOT NULL,
    SQL_Sequence FLOAT NOT NULL,
    SQL_Account_Name_Code VARCHAR NOT NULL,
    SQL_Account_Name VARCHAR NOT NULL,
    SQL_Account_Category_Order_Code VARCHAR NOT NULL,
    SQL_Account_Category_Order VARCHAR NOT NULL,
    SUB_Account_Category_Order_Code VARCHAR NULL,
    SUB_Account_Category_Order VARCHAR NULL,
    SQL_Account_Group_Name_Code FLOAT NULL,
    SQL_Account_Group_Name FLOAT NULL,
    Accountnumber_ID VARCHAR NOT NULL,
    January FLOAT NOT NULL,
    February FLOAT NOT NULL,
    March FLOAT NOT NULL,
    April FLOAT NOT NULL,
    May FLOAT NOT NULL,
    June FLOAT NOT NULL,
    July FLOAT NULL,
    August FLOAT NULL,
    September FLOAT NULL,
    October FLOAT NULL,
    November FLOAT NULL,
    December FLOAT NULL,
    Total FLOAT NULL,
    Account_Year INT

In [3]:
def create_database_from_csvs(csv_files, db_name='final_working_database.db'):
    """
    Create a SQLite database from multiple CSV files.
    Each CSV file will become a table in the database.
    
    Args:
        csv_files (list): List of CSV file paths
        db_name (str): Name of the SQLite database to create
    """
    import sqlite3
    
    try:
        # Create/connect to SQLite database
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()
        
        # Process each CSV file
        for csv_file in csv_files:
            # Get schema for the CSV
            create_table_sql = analyze_csv_schema(csv_file)
            
            # Execute create table statement
            cursor.execute(create_table_sql)
            
            # Read CSV data
            df = pd.read_csv(csv_file)
            
            # Get table name from CSV filename
            table_name = os.path.splitext(os.path.basename(csv_file))[0].upper()
            
            # Insert data into table
            df.to_sql(table_name, conn, if_exists='replace', index=False)
            
            print(f"Created table {table_name} and imported data from {csv_file}")
            
        conn.commit()
        print(f"\nDatabase {db_name} created successfully!")
        
    except Exception as e:
        print(f"Error creating database: {str(e)}")
        
    finally:
        conn.close()

# Example usage with multiple files:
csv_files = [
    'final_forecast_sheet.csv',
    'final_balance_sheet_tb_new.csv', 
    'final_balance_sheet_new.csv',
    'final_budget_sheet.csv',
    'final_income_sheet_new_seq.csv',
    'final_income_sheet_tb_new.csv'
    # Add more CSV files as needed
]
create_database_from_csvs(csv_files)

# You can also use glob to get all CSV files in a directory:
"""
import glob
csv_files = glob.glob('data/*.csv')  # Gets all CSVs from data directory
create_database_from_csvs(csv_files)
"""

Error creating database: table FINAL_FORECAST_SHEET already exists


"\nimport glob\ncsv_files = glob.glob('data/*.csv')  # Gets all CSVs from data directory\ncreate_database_from_csvs(csv_files)\n"